In [4]:
import lxml.html 
import pandas as pd
import re
import json
import os
from bs4 import BeautifulSoup as soup

# similarweb
* files are in a folder named 'datasimilar'
* output JSON files are saved in 'datascraped_similar'

In [63]:
folderName = 'datasimilar'
saveFolderName = 'datascraped_similar'
count = 0

# read all the saved files 
# put the information into pandas dataframe
for file in os.listdir(folderName):
    #print('-----------')
    if file.endswith(".html"):
        # dict object for json dumping
        companyJson = {}
        companyName = file[:-5]
        fileNameCompany = os.path.join(folderName, file)
        
        # 1. Open startup-list for city = cityname
        readSimilarWebFile = open(fileNameCompany, 'r')
        similarWebText = readSimilarWebFile.read()
        readSimilarWebFile.close()
        count = count + 1
        #print(companyName)
        similar_page_soup = soup(similarWebText, "html.parser")
        container_header = similar_page_soup.findAll("ul", {"class":"websiteHeader-companyInfoList is-collapsed js-websiteHeader-companyInfoList"})
        try:
            year_founded_str = container_header[0].p.getText()
            if (int(year_founded_str) > 1900 and int(year_founded_str) < 2019):
                companyJson['year_founded'] = int(year_founded_str)
            else:
                companyJson['year_founded'] = ''
        except:
            companyJson['year_founded'] = ''

        container = similar_page_soup.findAll("div", {"class":"websiteRanks-valueContainer js-websiteRanksValue"})
        try:
            companyJson['global_rank'] = int(container[0].getText().replace(' ', '').replace('\n', '').replace(',', ''))
            companyJson['country_rank'] = int(container[1].getText().replace(' ', '').replace('\n', '').replace(',', ''))
            companyJson['category_rank'] = int(container[2].getText().replace(' ', '').replace('\n', '').replace(',', ''))
        except:
            companyJson['global_rank'] = ''
            companyJson['country_rank'] = ''
            companyJson['category_rank'] = ''


        container_web_presence = similar_page_soup.findAll("div", {"class":"websitePage-engagementInfo"})
        web_presence = container_web_presence[0].findAll("span", {"class":"engagementInfo-valueNumber js-countValue"})
        if (len(web_presence) == 4):
            try:
                total_visit_str = web_presence[0].getText()
                if (total_visit_str.endswith('K')):
                    companyJson['total_visit'] = str(float(total_visit_str.replace('K', ''))*1000)
                elif (total_visit_str.endswith('M')):
                    companyJson['total_visit'] = str(float(total_visit_str.replace('M', ''))*1000000)
                elif (total_visit_str.endswith('B')):
                    companyJson['total_visit'] = str(float(total_visit_str.replace('B', ''))*1000000000)
                else: 
                    companyJson['total_visit'] = total_visit_str
                avg_vis_dur_str = web_presence[1].getText().split(':')
                avg_vis_dur_in_min = int(avg_vis_dur_str[0])*60 + int(avg_vis_dur_str[1]) + int(avg_vis_dur_str[2])/60 
                companyJson['avg_visit_duration_minute'] = float(avg_vis_dur_in_min)
                companyJson['pages_per_visit'] = float(web_presence[2].getText())
                companyJson['bounce_rate'] = float(web_presence[3].getText().replace('%', ''))
            except:
                companyJson['total_visit'] = ''
                companyJson['avg_visit_duration_minute'] = ''
                companyJson['pages_per_visit'] = ''
                companyJson['bounce_rate'] = ''
        else:
            companyJson['total_visit'] = ''
            companyJson['avg_visit_duration_minute'] = ''
            companyJson['pages_per_visit'] = ''
            companyJson['bounce_rate'] = ''

                
        savejson = saveFolderName + '/' + companyName + '.json'
        with open(savejson, 'w') as outfile:
            json.dump(companyJson, outfile)
                

       
        

# crunchbase

* files are in a folder named 'datacrunch'
* output JSON files are saved in 'datascraped_crunch'

In [248]:
# function for refromat the json
def formatJson(companyJson):
    #companyJson['Operating Status'] = ''
    try:
        full_date = companyJson['Founded Date']
        if (len(full_date.replace(' ', '').split(',')) > 1):
            fyear = full_date.replace(' ', '').split(',')[1]
            companyJson['Founded Date'] = fyear
    except:
        pass
    #companyJson['Funding Status'] = ''
    #companyJson['Last Funding Type'] = ''
    try:
        companyJson['Number of Employees'] = companyJson['Number of Employees'].replace(' ', '').replace('+', '').split('-')[1]
    except:
        pass
    #companyJson['Number of Acquisitions'] = ''
    #companyJson['Number of Investments'] = ''
    #companyJson['Number of Funding Rounds'] = ''
    try:
        tempstr = companyJson['Total Funding Amount'].replace('$', '').replace('CAD', '').replace('CA', '')
        if (tempstr.endswith('K')):
            companyJson['Total Funding Amount'] = str(float(tempstr.replace('K', ''))*1000)
        elif (total_visit_str.endswith('M')):
            companyJson['Total Funding Amount'] = str(float(tempstr.replace('M', ''))*1000000)
        elif (total_visit_str.endswith('B')):
            companyJson['Total Funding Amount'] = str(float(tempstr.replace('B', ''))*1000000000)
        else:
            companyJson['Total Funding Amount'] = str(float(tempstr))
    except:
        pass
    #companyJson['Number of Lead Investors'] = ''
    #companyJson['Number of Investors'] = ''
    #companyJson['Number of Exits'] = ''
    #companyJson['Number of Current Team Members'] = ''
    #companyJson['Number of Board Members / Advisors'] = ''
    #companyJson['Number of Sub-Orgs'] = ''
    try:
        companyJson['CB Rank (Company)'] = int(companyJson['CB Rank (Company)'].replace(',', ''))
    except:
        pass
    #companyJson['Number of Events'] = ''
    return companyJson
    

In [247]:
folderName = 'datacrunch'
saveFolderName = 'datascraped_crunch'
count = 0


# read all the saved files 
# put the information into pandas dataframe
for file in os.listdir(folderName):
    #print('-----------')
    if file.endswith(".html"):
        # dict object for json dumping INITIALIZE
        companyJson = {}
        companyJson['Operating Status'] = ''
        companyJson['Founded Date'] = ''
        companyJson['Funding Status'] = ''
        companyJson['Last Funding Type'] = ''
        companyJson['Number of Employees'] = ''
        companyJson['Number of Acquisitions'] = ''
        companyJson['Number of Investments'] = ''
        companyJson['Number of Funding Rounds'] = ''
        companyJson['Total Funding Amount'] = ''
        companyJson['Number of Lead Investors'] = ''
        companyJson['Number of Investors'] = ''
        companyJson['Number of Exits'] = ''
        companyJson['Number of Current Team Members'] = ''
        companyJson['Number of Board Members / Advisors'] = ''
        companyJson['Number of Sub-Orgs'] = ''
        companyJson['CB Rank (Company)'] = ''
        companyJson['Number of Events'] = ''
        # --------------------------
        companyName = file[:-5]
        fileNameCompany = os.path.join(folderName, file)
        
        # 1. Open startup-list for city = cityname
        readCrunchWebFile = open(fileNameCompany, 'r')
        crunchWebText = readCrunchWebFile.read()
        readCrunchWebFile.close()
        count = count + 1
        #print(companyName)
        crunch_page_soup = soup(crunchWebText, "html.parser")
        
        x = crunch_page_soup.findAll("div", {"class":"layout-wrap layout-row"})
        my_list = re.sub(r'\n+', '\n', x[0].text).replace('  ', '').replace('\xa0', '').replace('\n\n', '\n').split('\n')[1:]
        i = 0
        while i < len(my_list):
            if (my_list[i] == 'Operating Status' or
               my_list[i] == 'Founded Date' or
               my_list[i] == 'Funding Status' or
               my_list[i] == 'Last Funding Type' or
               my_list[i] == 'Number of Employees' or
               my_list[i] == 'Founded Date'):
                companyJson[my_list[i]] = my_list[i + 1]
            i = i + 1
            
        y = crunch_page_soup.findAll("div", {"class":"flex-100 flex-gt-sm-50 bigValueItem layout-column ng-star-inserted"})
        for i in range(len(y)):
            my_list = re.sub(r'\n+', '\n', y[i].text).replace('  ', '').replace('\xa0', '').replace('\n\n', '\n').split('\n')[1:]
            if (my_list[0] == 'Number of Acquisitions' or
                my_list[0] == 'Number of Investments' or
                my_list[0] == 'Number of Funding Rounds' or
                my_list[0] == 'Total Funding Amount' or
                my_list[0] == 'Number of Lead Investors' or
                my_list[0] == 'Number of Investors' or
                my_list[0] == 'Number of Exits' or
                my_list[0] == 'Number of Current Team Members' or
                my_list[0] == 'Number of Board Members / Advisors' or
                my_list[0] == 'Number of Sub-Orgs' or
                my_list[0] == 'CB Rank (Company)' or
                my_list[0] == 'Number of Events'):
                companyJson[my_list[0]] = my_list[1]
                
        formatedjson = formatJson(companyJson)
        savejson = saveFolderName + '/' + companyName + '.json'
        with open(savejson, 'w') as outfile:
            json.dump(formatedjson, outfile)
